<a href="https://colab.research.google.com/github/Murad-pitafi/Computer-Vision/blob/main/Data_Generation_and_Latent_Space_Exploration_with_Variational_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torch import nn, optim
from torch.autograd import Variable
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.10MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.09MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.24MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class VAE(nn.Module):
    def __init__(self, latent_dim=2):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 400), nn.ReLU(), nn.Linear(400, 128), nn.ReLU())
        self.mu_layer = nn.Linear(128, latent_dim)
        self.logvar_layer = nn.Linear(128, latent_dim)
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 128), nn.ReLU(), nn.Linear(128, 400), nn.ReLU(), nn.Linear(400, 28 * 28), nn.Sigmoid())

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = self.mu_layer(h), self.logvar_layer(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 28 * 28))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [4]:
vae = VAE()
optimizer = optim.Adam(vae.parameters(), lr=0.001)

def loss_function(recon_x, x, mu, logvar):
    recon_loss = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 28 * 28), reduction='sum')
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld


In [6]:
epochs = 10
for epoch in range(epochs):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {train_loss/len(train_loader.dataset):.4f}')


In [6]:
def visualize_reconstructions():
    vae.eval()
    data, _ = next(iter(train_loader))
    data = data[:8]
    recon, _, _ = vae(Variable(data))
    fig, axes = plt.subplots(2, 8, figsize=(15, 4))
    for i in range(8):
        axes[0, i].imshow(data[i].view(28, 28).detach().numpy(), cmap='gray')
        axes[1, i].imshow(recon[i].view(28, 28).detach().numpy(), cmap='gray')
        axes[0, i].axis('off')
        axes[1, i].axis('off')
    plt.show()

visualize_reconstructions()


In [ ]:
z_samples = torch.randn(16, 2)
generated_images = vae.decode(z_samples).detach()
fig, axes = plt.subplots(1, 16, figsize=(15, 4))
for i, img in enumerate(generated_images):
    axes[i].imshow(img.view(28, 28).numpy(), cmap='gray')
    axes[i].axis('off')
plt.show()


In [ ]:
import numpy as np
z1 = np.linspace(-3, 3, 10)
z2 = np.linspace(-3, 3, 10)
fig, axes = plt.subplots(10, 10, figsize=(10, 10))
for i, z1_val in enumerate(z1):
    for j, z2_val in enumerate(z2):
        z = torch.FloatTensor([[z1_val, z2_val]])
        generated_img = vae.decode(z).detach()
        axes[i, j].imshow(generated_img.view(28, 28).numpy(), cmap='gray')
        axes[i, j].axis('off')
plt.show()
